# JSON数据分析 - 筛选包含'no response generated'的文件

本notebook用于分析实验数据目录中的JSON文件，筛选出包含'no response generated'字段的文件并统计数量。

**数据路径**: `/home/dyvm6xra/dyvm6xrauser44/wujiamin/AgenticRAG/rag_evals/results/aip_rag_perf/old/experiment_20251211_113947/processed_json`

## 1. 导入所需库

In [ ]:
import os
import json
import pandas as pd
from pathlib import Path

## 2. 定义数据路径

In [ ]:
# 设置实验数据目录路径
base_dir = "/home/dyvm6xra/dyvm6xrauser44/wujiamin/AgenticRAG/rag_evals/results/aip_rag_perf/old/experiment_20251211_113947"
processed_json_dir = os.path.join(base_dir, "processed_json")

# 检查路径是否存在
if os.path.exists(processed_json_dir):
    print(f"✓ 数据目录存在: {processed_json_dir}")
    # 列出目录中的所有文件
    json_files = [f for f in os.listdir(processed_json_dir) if f.endswith('.json')]
    print(f"✓ 找到 {len(json_files)} 个JSON文件")
else:
    print(f"✗ 数据目录不存在: {processed_json_dir}")

## 3. 定义搜索函数

定义一个函数来递归搜索JSON中所有包含特定字符串的位置

In [ ]:
def contains_no_response(data, search_string="no response generated"):
    """
    递归搜索JSON数据中是否包含指定字符串（不区分大小写）
    
    参数:
        data: JSON数据（dict, list, str等）
        search_string: 要搜索的字符串
    
    返回:
        bool: 如果找到则返回True，否则返回False
    """
    search_lower = search_string.lower()
    
    if isinstance(data, dict):
        for value in data.values():
            if contains_no_response(value, search_string):
                return True
    elif isinstance(data, list):
        for item in data:
            if contains_no_response(item, search_string):
                return True
    elif isinstance(data, str):
        if search_lower in data.lower():
            return True
    
    return False


def count_string_occurrences(data, search_string="no response generated"):
    """
    递归统计JSON数据中指定字符串出现的次数（不区分大小写）
    
    参数:
        data: JSON数据（dict, list, str等）
        search_string: 要搜索的字符串
    
    返回:
        int: 字符串出现的次数
    """
    search_lower = search_string.lower()
    count = 0
    
    if isinstance(data, dict):
        for value in data.values():
            count += count_string_occurrences(value, search_string)
    elif isinstance(data, list):
        for item in data:
            count += count_string_occurrences(item, search_string)
    elif isinstance(data, str):
        if search_lower in data.lower():
            count += 1
    
    return count

## 4. 分析所有JSON文件

遍历所有JSON文件，找出包含'no response generated'的文件

In [ ]:
# 存储包含'no response generated'的文件列表
files_with_no_response = []

# 遍历所有JSON文件
for json_file in json_files:
    file_path = os.path.join(processed_json_dir, json_file)
    
    try:
        # 读取JSON文件
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        
        # 检查是否包含'no response generated'
        if contains_no_response(data):
            files_with_no_response.append(json_file)
            print(f"✓ {json_file} - 包含 'no response generated'")
        else:
            print(f"  {json_file} - 不包含")
            
    except Exception as e:
        print(f"✗ 读取文件出错 {json_file}: {str(e)}")

print("\n" + "="*60)

## 6. 基本统计结果（至少出现1次）

## 5. 按出现次数筛选文件

统计每个文件中字符串的出现次数，并筛选出现次数超过k次的文件

In [ ]:
# 设置参数
search_string = "no response generated"  # 要搜索的字符串
min_occurrences = 1  # 最小出现次数阈值（k值）

# 统计每个文件中字符串的出现次数
file_occurrence_counts = {}

print(f"正在统计 '{search_string}' 的出现次数...\n")
print("="*60)

for json_file in json_files:
    file_path = os.path.join(processed_json_dir, json_file)
    
    try:
        # 读取JSON文件
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        
        # 统计出现次数
        count = count_string_occurrences(data, search_string)
        file_occurrence_counts[json_file] = count
        
        if count > 0:
            print(f"📄 {json_file}: {count} 次")
        else:
            print(f"   {json_file}: 0 次")
            
    except Exception as e:
        print(f"✗ 读取文件出错 {json_file}: {str(e)}")
        file_occurrence_counts[json_file] = 0

print("="*60)

In [ ]:
# 筛选出现次数超过k次的文件
files_above_threshold = {
    filename: count 
    for filename, count in file_occurrence_counts.items() 
    if count > min_occurrences
}

# 按出现次数降序排序
sorted_files = sorted(files_above_threshold.items(), key=lambda x: x[1], reverse=True)

print(f"\n📊 筛选结果 (出现次数 > {min_occurrences}):")
print(f"   符合条件的文件数: {len(sorted_files)}")
print(f"\n文件列表（按出现次数降序）:")
print("="*60)

for i, (filename, count) in enumerate(sorted_files, 1):
    print(f"{i:2d}. {filename}: {count} 次")

if len(sorted_files) == 0:
    print(f"   没有文件的出现次数超过 {min_occurrences} 次")

## 5.1 数据可视化（可选）

使用pandas DataFrame展示统计结果

In [ ]:
# 创建DataFrame用于数据展示
df_stats = pd.DataFrame([
    {"文件名": filename, "出现次数": count} 
    for filename, count in file_occurrence_counts.items()
])

# 按出现次数降序排序
df_stats = df_stats.sort_values("出现次数", ascending=False).reset_index(drop=True)

# 添加序号列
df_stats.index = df_stats.index + 1

print("\n完整统计表格:")
print("="*80)
display(df_stats)

# 统计信息
print(f"\n汇总统计:")
print(f"  总文件数: {len(df_stats)}")
print(f"  包含目标字符串的文件数: {len(df_stats[df_stats['出现次数'] > 0])}")
print(f"  不包含目标字符串的文件数: {len(df_stats[df_stats['出现次数'] == 0])}")
print(f"  总出现次数: {df_stats['出现次数'].sum()}")
print(f"  平均出现次数: {df_stats['出现次数'].mean():.2f}")
print(f"  最大出现次数: {df_stats['出现次数'].max()}")

In [ ]:
# 统计结果
total_files = len(json_files)
files_with_issue = len(files_with_no_response)

print(f"📊 统计结果:")
print(f"   总文件数: {total_files}")
print(f"   包含'no response generated'的文件数: {files_with_issue}")
print(f"   百分比: {(files_with_issue/total_files*100):.2f}%")
print("\n包含'no response generated'的文件列表:")
for i, filename in enumerate(files_with_no_response, 1):
    print(f"   {i}. {filename}")

## 7. 详细分析（可选）

查看每个文件中具体哪些字段包含'no response generated'

In [ ]:
def find_no_response_paths(data, search_string="no response generated", path=""):
    """
    递归搜索JSON数据，返回包含指定字符串的所有路径
    
    参数:
        data: JSON数据
        search_string: 要搜索的字符串
        path: 当前路径
    
    返回:
        list: 包含匹配字符串的路径列表
    """
    search_lower = search_string.lower()
    paths = []
    
    if isinstance(data, dict):
        for key, value in data.items():
            new_path = f"{path}.{key}" if path else key
            paths.extend(find_no_response_paths(value, search_string, new_path))
    elif isinstance(data, list):
        for i, item in enumerate(data):
            new_path = f"{path}[{i}]"
            paths.extend(find_no_response_paths(item, search_string, new_path))
    elif isinstance(data, str):
        if search_lower in data.lower():
            paths.append(path)
    
    return paths

# 对每个包含问题的文件进行详细分析
print("\n" + "="*60)
print("详细分析 - 查看具体位置:")
print("="*60 + "\n")

for json_file in files_with_no_response:
    file_path = os.path.join(processed_json_dir, json_file)
    
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    paths = find_no_response_paths(data)
    print(f"\n📄 {json_file}")
    print(f"   找到 {len(paths)} 个位置包含 'no response generated':")
    for i, path in enumerate(paths[:5], 1):  # 只显示前5个
        print(f"   {i}. {path}")
    if len(paths) > 5:
        print(f"   ... 还有 {len(paths)-5} 个位置")